# Permutation tests


# Homework

## 5.1 Baseline for linear prediction* (<font color='red'>2p</font>)

Take some benchmark dataset for regression where at least some features are discrete.
* Train a linear predictor. Estimate the training error. 
* Train a linear predictor on the data where all target values are permuted. Estimate test error on the original dataset. 
* Train a linear predictor on the data where target values are permuted inside the group that is defined for some discrete feature. Estimate the test error on the original dataset. 

Repeat permutations so that you get distributions for test error. Estimate where the original training error is located inside these distributions and interpret results. You can also compare distributions of test errors for two permutation schemes. What does the difference tell you. 
